In [2]:
# Importing required Libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from pywaffle import Waffle
import random
# we are using the inline backend
%matplotlib inline 

In [3]:
# Reading dataset

df = pd.read_csv('data_analysis/dataset/no_null_df.csv')
df.head()

,Unnamed: 0,Make,Model,Variant,Ex-Showroom_Price(Rs),Displacement(cc),Cylinders,Valves_Per_Cylinder,Drivetrain,Cylinder_Configuration,...,iPod_Compatibility,ESP_(Electronic_Stability_Program),Cooled_Glove_Box,Turbocharger,ISOFIX_(Child-Seat_Mount),Rain_Sensing_Wipers,Leather_Wrapped_Steering,Automatic_Headlamps,ASR_Traction_Control,Cruise_Control
0,0,Tata,Nano Genx,Xt,292667,624.0,2.0,2.0,RWD (Rear Wheel Drive),In-line,...,False,False,False,False,False,False,False,False,False,False
1,1,Tata,Nano Genx,Xe,236447,624.0,2.0,2.0,RWD (Rear Wheel Drive),In-line,...,False,False,False,False,False,False,False,False,False,False
2,2,Tata,Nano Genx,Emax Xm,296661,624.0,2.0,2.0,RWD (Rear Wheel Drive),In-line,...,False,False,False,False,False,False,False,False,False,False
3,3,Tata,Nano Genx,Xta,334768,624.0,2.0,2.0,RWD (Rear Wheel Drive),In-line,...,False,False,False,False,False,False,False,False,False,False
4,4,Tata,Nano Genx,Xm,272223,624.0,2.0,2.0,RWD (Rear Wheel Drive),In-line,...,False,False,False,False,False,False,False,False,False,False


In [4]:
df.rename(columns={'ARAI_Certified_Mileage(km/litre)':'ARAI_Certified_Mileage(km_per_litre)',
'City_Mileage(km/litre)':'City_Mileage(km_per_litre)',
'Highway_Mileage(km/litre)':'Highway_Mileage(km_per_litre)'},inplace=True)

In [5]:
# not using all columns for practical reasons
# # Creating list of columns with type object
# object_columns=df.select_dtypes(include=['object']).columns
# object_columns=object_columns.tolist()
# # print(object_columns)

# # Creating list of columns with type boolean
# bool_columns=df.select_dtypes(include=['bool']).columns
# bool_columns=bool_columns.tolist()
# # print(bool_columns)

# # Creating list of columns with type float
# float_columns=df.select_dtypes(include=['float']).columns
# float_columns=float_columns.tolist()
# # print(float_columns)

In [6]:
#relevent columns
object_columns=df[['Make','Drivetrain','Emission_Norm','Fuel_Type','Body_Type','Gears','Front_Brakes','Rear_Brakes','Power_Steering','Power_Windows','Keyless_Entry','Type','Third_Row_AC_Vents','Ventilation_System','Parking_Assistance']]
bool_columns=df[['Central_Locking','Auto-Dimming_Rear-View_Mirror','Navigation_System','Second_Row_AC_Vents','Rain_Sensing_Wipers']]
float_columns=df[['Ex-Showroom_Price(Rs)','Cylinders','Height(mm)','Length(mm)','Width(mm)','City_Mileage(km_per_litre)','Ground_Clearance(mm)','Seating_Capacity','Wheelbase','Minimum_Turning_Radius(meter)','Number_of_Airbags']]

In [7]:

# Adding a column with all values same in the dataframe
df["Constant_Column"] = 1

# Creating a list which will contain description, variables and required details of all kinds of plots
data_vis=[]

In [8]:
#WAFFLE for all booleans

for i in bool_columns :
  df_test=df[[i,'Constant_Column']]
  df_grp = df_test.groupby([i],as_index=False).count()

  total=df_grp['Constant_Column'][0]+df_grp['Constant_Column'][1]
  n_true=df_grp['Constant_Column'][1]

  # Adding data for this plot
  data_vis.append({
      'Type':'WAFFLE',
      'variable_1':i,
      'Description':i+' feature exists in '+str(round(n_true/total,2))+' percent of Cars.',
      'path': './static/img/WAFFLE/'+str(i)+'.png'
  })

  # Plotting Waffle Chart
  fig = plt.figure(
      FigureClass=Waffle, 
      rows=25, 
      values=list(df_grp['Constant_Column']),
      labels=list(df_grp[i]),
      title={
        'label': i,
      },
      figsize=(12, 8),
      legend={'bbox_to_anchor': (0.5, 0.5)}    
  )

  # Saving the plot
  plt.savefig('./static/img/WAFFLE/'+str(i)+'.png')

  # displaying plot
  # plt.show()
  plt.close()

In [9]:

# BOX-Plots for all floats

for i in float_columns :
  plt.rcParams['figure.figsize'] = (12,8)
  ax=sns.boxplot(x=df['Constant_Column'],y=df[i])
  plt.xlabel('')
  plt.title(i)
  ax.axes.xaxis.set_visible(False)

  # Saving the plot
  plt.savefig('./static/img/BOX/'+str(i)+'.png')

  # Adding data for this plot
  data_vis.append({
      'Type':'BOX',
      'variable_1':i,
      'Description':i+' has a mean value of '+str(round(df[i].mean(),2))+', min and max value of '+str(df[i].min())+' and '+str(df[i].max())+', respectively. Its InterQuartile Range (IQR) is '+str(round(df[i].quantile(q=0.75),2)-round(df[i].quantile(q=0.25),2))+'.',
      'path': './static/img/BOX/'+str(i)+'.png'
  })

  # displaying plot
  # plt.show()
  plt.close()



In [10]:
# PIE CHARTS for all objects with less than 10 unique items

for i in object_columns :
  if(df[i].nunique()<10):
    palette_color = sns.color_palette('bright')
    df_test=df[[i,'Constant_Column']]
    df_grp = df_test.groupby([i],as_index=False).count()
    df_grp_list=df_grp[i].tolist()

    c_max=df_grp['Constant_Column'].max()
    c_min=df_grp['Constant_Column'].min()

    hper=df_grp[df_grp['Constant_Column']==c_max][i].tolist()[0]
    lper=df_grp[df_grp['Constant_Column']==c_min][i].tolist()[0]
    # Adding data for this plot
    data_vis.append({
      'Type':'PIE',
      'variable_1':i,
      'Description':'Among '+i+', '+hper+' and '+lper+' has highest and lowest percentage, respectively',
      'path': './static/img/PIE/'+str(i)+'.png'
    })
    # function for displaying percent value and the count 
    def func(pct, allvalues):
      absolute = int(pct / 100.*np.sum(allvalues))
      return "{:.1f}%\n({:d} )".format(pct, absolute)

    # plotting data on chart
    plt.pie(df_grp['Constant_Column'],labels=df_grp_list, colors=palette_color, autopct = lambda pct: func(pct, df_grp['Constant_Column']))
    plt.title('PIE-CHART for '+i)

    # Saving the plot
    plt.savefig('./static/img/PIE/'+str(i)+'.png')

    # displaying chart
    # plt.show()
    plt.close()

In [11]:
# BOX PLOTS for objects v/s floats

for i in object_columns :
  for j in float_columns:
    if df[i].nunique()<=39:

      ax=sns.boxplot(x=df[i],y=df[j])
      plt.title(i+' v/s '+j)
      plt.xticks(rotation=90)
      df_test=df[[i,j]]

      df_grp_mean = df_test.groupby([i],as_index=False).mean()
      df_grp_25p = df_test.groupby([i],as_index=False).quantile(q=0.25)
      df_grp_75p = df_test.groupby([i],as_index=False).quantile(q=0.75)

      df_iqr=df_grp_75p.set_index(i).subtract(df_grp_25p.set_index(i),fill_value=0)

      max_iqr_o=df_iqr[df_iqr[j]==df_iqr[j].max()].index.tolist()[0]
      min_iqr_o=df_iqr[df_iqr[j]==df_iqr[j].min()].index.tolist()[0]

      max_mean_o=df_grp_mean[df_grp_mean[j]==df_grp_mean[j].max()][i].tolist()[0]
      min_mean_o=df_grp_mean[df_grp_mean[j]==df_grp_mean[j].min()][i].tolist()[0]

      # Adding data for this plotv
      data_vis.append({
        'Type':'BOX',
        'variable_1':i,
        'variable_2':j,
        'Description':'In '+i+'s comparison wrt '+j+','+max_mean_o+' has the highest mean whereas, '+min_mean_o+' has the lowest mean, max and min InterQuartile Range (IQR) is shown by '+max_iqr_o+' and '+min_iqr_o+' respectively.',
        'path': './static/img/BOX/'+str(i)+'_'+str(j)+'.png'
      })

      # Saving the plot
      plt.savefig('./static/img/BOX/'+str(i)+'_'+str(j)+'.png')

      # displaying plot
      # plt.show()
      plt.close()

In [12]:
# REG PLOTS for floats v/s floats

for i in float_columns:
  for j in float_columns:
    if i!= j :
      sns.regplot(x = i, 
              y = j, 
              data = df)
      plt.title('Reg-Plot for '+i+' and '+j)  
      
      if(np.polyfit(df[i],df[j],1)[0]>0):
        # Saving the plot
        plt.savefig('./static/img/REG_PLOT/positive/'+str(i)+'_'+j+'.png')
        # Adding data for this plot
        data_vis.append({
        'Type':'REG_PLOT',
        'variable_1':i,
        'variable_2':j,
        'Description':i+' vs '+j+' has a positive corelation. The slope and intercept are '+str(round(np.polyfit(df[i],df[j],1)[0],1))+' and '+str(round(np.polyfit(df[i],df[j],1)[1],2))+' , respectively',
        'path': './static/img/REG_PLOT/positive/'+str(i)+'_'+j+'.png'
        })
      else:
        # Saving the plot
        plt.savefig('./static/img/REG_PLOT/negative/'+str(i)+'_'+j+'.png')
        # Adding data for this plot
        data_vis.append({
        'Type':'REG_PLOT',
        'variable_1':i,
        'variable_2':j,
        'Description':i+' vs '+j+' has a negative corelation. The slope and intercept are '+str(round(np.polyfit(df[i],df[j],1)[0],1))+' and '+str(round(np.polyfit(df[i],df[j],1)[1],2))+' , respectively',
        'path': './static/img/REG_PLOT/negative/'+str(i)+'_'+j+'.png'
        })
      # displaying plot
      # plt.show()
      plt.close() 

In [13]:
# Object v/s bool bar graph with count
df_btos = df.replace({True: 'TRUE', False: 'FALSE'})
for i in object_columns:
  for j in bool_columns:
    if df[i].nunique()<=39:
      df_test=df_btos[[i,j,'Constant_Column']]
      
      df_grp = df_test.groupby([i,j],as_index=False).count()

      n_max_true=df_grp[df_grp[j]=='TRUE']['Constant_Column'].max()
      max_true=df_grp[(df_grp[j]=='TRUE') & (df_grp['Constant_Column']==n_max_true)][i].tolist()[0]

      n_min_true=df_grp[df_grp[j]=='TRUE']['Constant_Column'].min()
      min_true=df_grp[(df_grp[j]=='TRUE') & (df_grp['Constant_Column']==n_min_true)][i].tolist()[0]

      # Adding data for this plot
      data_vis.append({
        'Type':'BAR',
        'variable_1':i,
        'variable_2':j,
        'Description':'Among '+i+' , '+j+' is available in most number of '+max_true+' and least available in '+min_true+'.',
        'path': './static/img/BAR/'+str(i)+'_'+str(j)+'.png'
        })
      ax = sns.countplot(x=df[i], hue=df_btos[j], data=df)
      plt.xticks(rotation=90)

      # Saving the plot
      plt.savefig('./static/img/BAR/'+str(i)+'_'+str(j)+'.png')

      # displaying plot
      # plt.show()
      plt.close()

In [14]:
# float vs boolean grouped low medium high bar graph

for i in float_columns :
  for j in bool_columns:
    df_btos = df.replace({True: 'TRUE', False: 'FALSE'})
    bins = np.linspace(min(df_btos[i]), max(df_btos[i]), 4)

    group_names = ['Low', 'Medium', 'High']

    df_btos[i+'-binned'] = pd.cut(df_btos[i], bins, labels=group_names, include_lowest=True )
    
    df_test=df_btos[[i+'-binned',j,'Constant_Column']]
    df_grp = df_test.groupby([i+'-binned',j],as_index=False).count()

    n_max_true=df_grp[df_grp[j]=='TRUE']['Constant_Column'].max()
    max_true=df_grp[(df_grp[j]=='TRUE') & (df_grp['Constant_Column']==n_max_true)][i+'-binned'].tolist()[0]

    n_max_false=df_grp[df_grp[j]=='FALSE']['Constant_Column'].max()
    max_false=df_grp[(df_grp[j]=='FALSE') & (df_grp['Constant_Column']==n_max_false)][i+'-binned'].tolist()[0]

    # Adding data for this plot
    data_vis.append({
        'Type':'BAR',
        'variable_1':i,
        'variable_2':j,
        'Description':'On dividing '+i+' into categories , '+j+' is readily available in '+max_true+' category and least available in '+min_true+' category.',
        'path': './static/img/BAR/'+str(i)+'_'+str(j)+'.png'
        })
    ax = sns.countplot(x=df_btos[i+'-binned'], hue=df_btos[j], data=df)
    plt.xticks(rotation=90)
    plt.xlabel(i)
    plt.ylabel("Count")

    # Saving the plot
    plt.savefig('./static/img/BAR/'+str(i)+'_'+str(j)+'.png')

    # displaying plot
    # plt.show()
    plt.close() 

In [15]:
# float vs boolean boxplot

for i in float_columns :
  for j in bool_columns:
    df_btos = df.replace({True: 'TRUE', False: 'FALSE'})
    ax=sns.boxplot(x=df[i],y=df_btos[j])
    plt.xlabel(i)
    plt.ylabel(j)
    df_test=df_btos[[j,i]]
    df_grp_mean = df_test.groupby([j],as_index=False).mean()
    true_mean= df_grp_mean[df_grp_mean[j]=='TRUE'][i].tolist()[0]
    false_mean=df_grp_mean[df_grp_mean[j]=='FALSE'][i].tolist()[0]

    # Adding data for this plot
    data_vis.append({
        'Type':'BOX',
        'variable_1':i,
        'variable_2':j,
        'Description':'For cars with feature '+j+',the mean '+i+' is '+str(true_mean)+' and for the rest of the cars mean '+i+' is '+str(false_mean)+'.' ,
        'path': './static/img/BOX/'+str(i)+'_'+str(j)+'.png'
        })

    plt.title(i+' v/s '+j)
    plt.xticks(rotation=90)

    # Saving the plot
    plt.savefig('./static/img/BOX/'+str(i)+'_'+str(j)+'.png')

    # displaying plot
    # plt.show()
    plt.close() 

In [19]:
# HEATMAP for all two combinations of object columns whose number of unique entry is less than 20

for i in object_columns:
  for j in object_columns:
    if i!=j and df[i].nunique()<=20 and df[j].nunique()<=20 :
      df_test=df[[i,j,'Constant_Column']]
      df_grp = df_test.groupby([i,j],as_index=False).count()
      max_x=df_grp[df_grp['Constant_Column']==df_grp['Constant_Column'].max()][i].tolist()[0]
      max_y=df_grp[df_grp['Constant_Column']==df_grp['Constant_Column'].max()][j].tolist()[0]

      # Adding data for this plot
      data_vis.append({
        'Type':'HEATMAP',
        'variable_1':i,
        'variable_2':j,
        'Description':'The most popular combination of '+i+' and '+j+' is '+max_x+' and '+max_y+'.',
        'path': './static/img/HEATMAP/'+str(i)+'_'+str(j)+'.png'
        })

      df_pivot=df_grp.pivot(index=i,columns=j)

      fig, ax = plt.subplots()
      im = ax.pcolor(df_pivot, cmap='RdBu')

      row_labels = df_pivot.columns.levels[1]
      col_labels = df_pivot.index

      ax.set_xticks(np.arange(df_pivot.shape[1]) + 0.5, minor=False)
      ax.set_yticks(np.arange(df_pivot.shape[0]) + 0.5, minor=False)

      ax.set_xticklabels(row_labels, minor=False)
      ax.set_yticklabels(col_labels, minor=False)
      plt.title('')
      plt.ylabel(i)
      plt.xlabel(j)

      plt.xticks(rotation=90)

      fig.colorbar(im)

      # Saving the plot
      plt.savefig('./static/img/HEATMAP/'+i+'_'+j+'.png')

      # displaying plot
      # plt.show()
      plt.close()

In [20]:
# Waffle chart for two booleans
for i in bool_columns:
  for j in bool_columns:
    if i!=j:
      df_test=df[[i,j,'Constant_Column']]
      df_grp = df_test.groupby([i,j],as_index=False).count()
      index_=['Neither', j, i, i+' and '+j]
      df_grp.index=index_

      # Adding data for this plot
      data_vis.append({
        'Type':'WAFFLE',
        'variable_1':i,
        'variable_2':j,
        'Description':'This graph shows how frequently '+i+' and '+j+' features are seen together, independent of each other or not at all.',
        'path': './static/img/WAFFLE/'+i+'_'+j+'.png'
        })
      fig = plt.figure(
          FigureClass=Waffle, 
          rows=25, 
          values=list(df_grp['Constant_Column']),
          labels=list(df_grp.index),
          title={
              'label': i+" and "+j,
          },
          figsize=(12, 8),
          legend={'bbox_to_anchor': (0.5, 0.5)}    
      )

      # Saving the plot
      plt.savefig('./static/img/WAFFLE/'+i+'_'+j+'.png')

      # displaying plot
      # plt.show()
      plt.close()

In [21]:
# Scatter plots for two float cplumns

for i in float_columns:
  for j in float_columns:
    if i!=j:
      plt.scatter(df[i], df[j])
      plt.xlabel(i)
      plt.ylabel(j)
      plt.title(i+' v/s '+j)

      # Adding data for this plot
      data_vis.append({
        'Type':'SCATTER',
        'variable_1':i,
        'variable_2':j,
        'Description':'This graph shows the distribution between '+i+' and '+j+'.',
        'path': './static/img/SCATTER/'+i+'_'+j+'.png'
        })
      
      # Saving the plot
      plt.savefig('./static/img/SCATTER/'+i+'_'+j+'.png')

      # displaying plot
      # plt.show()
      plt.close()

In [22]:
pd.DataFrame(data_vis)

,Type,variable_1,Description,path,variable_2
0,WAFFLE,Central_Locking,Central_Locking feature exists in 0.88 percent...,./static/img/WAFFLE/Central_Locking.png,NaN
1,WAFFLE,Auto-Dimming_Rear-View_Mirror,Auto-Dimming_Rear-View_Mirror feature exists i...,./static/img/WAFFLE/Auto-Dimming_Rear-View_Mir...,NaN
2,WAFFLE,Navigation_System,Navigation_System feature exists in 0.54 perce...,./static/img/WAFFLE/Navigation_System.png,NaN
3,WAFFLE,Second_Row_AC_Vents,Second_Row_AC_Vents feature exists in 0.53 per...,./static/img/WAFFLE/Second_Row_AC_Vents.png,NaN
4,WAFFLE,Rain_Sensing_Wipers,Rain_Sensing_Wipers feature exists in 0.37 per...,./static/img/WAFFLE/Rain_Sensing_Wipers.png,NaN
...,...,...,...,...,...
781,SCATTER,Number_of_Airbags,This graph shows the distribution between Numb...,./static/img/SCATTER/Number_of_Airbags_City_Mi...,City_Mileage(km_per_litre)
782,SCATTER,Number_of_Airbags,This graph shows the distribution between Numb...,./static/img/SCATTER/Number_of_Airbags_Ground_...,Ground_Clearance(mm)
783,SCATTER,Number_of_Airbags,This graph shows the distribution between Numb...,./static/img/SCATTER/Number_of_Airbags_Seating...,Seating_Capacity
784,SCATTER,Number_of_Airbags,This graph shows the distribution between Numb...,./static/img/SCATTER/Number_of_Airbags_Wheelba...,Wheelbase


In [26]:
vdf=pd.DataFrame(data_vis)
vdf.to_csv('data_analysis/dataset/data_vis.csv')